##### Определение списка договоров на каждую дату decision_date и ИНН контрагента

In [1]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки

In [2]:
# сюда таблицы служебные будем кидать, для отчетов
name = pre + "data/interm/auxilary/feature_contract_cnt.xlsx"
writer = ExcelWriter(name)

In [3]:
name = pre + "data/interm/target_data.pkl"
target_data = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_data))
# target_data.head(1)

127968


In [4]:
name = pre + "data/interm/external_pkl/portfolio.pkl"
portfolio = pd.read_pickle(name, compression={'method':'zip'})

print(len(portfolio))
# portfolio.head(1)

268031


In [5]:
portfolio['open_auxilary'] = portfolio['REHIRING_DT'].combine_first(portfolio['OPEN_DT'])

merge = pd.merge(target_data, portfolio, how='inner', left_on='ИНН контрагента', right_on='INN')

mask_open = (merge['Дата принятия решения по сделке']>merge['open_auxilary'])

# timedelta(days=1) - для тестирования, так как этих данных еще не было на prod, в рабочей версии делать не надо 
mask_leasing = (merge['Дата принятия решения по сделке']>merge['LEASE_DATE']+timedelta(days=1))

# без тех, что закончились более 5 лет назад
mask_closed = (merge['Дата принятия решения по сделке']<=merge['CLOSE_DT']+timedelta(days=1825)) # 365*5

last_contracts = merge[mask_open&mask_closed&mask_leasing].copy()

# last_contracts.head(1)

In [6]:
keep_col = ['CONTRACT_KEY', 'CONTRACT_APP_KEY', 'CONTRACT_NUM', 'ИНН контрагента', 'decision_date', 'CLOSE_DT', 'open_auxilary', 'LEASE_DATE']

target_ctr = pd.merge(target_data, last_contracts[keep_col], how='left', on=['ИНН контрагента', 'decision_date'], suffixes=['', 'lst'])

# target_ctr.head(1)

In [7]:
target_ctr.to_pickle(pre+"data/interm/lst_contracts.pkl", compression={'method': 'zip'})

In [8]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save() 